Testing data positive feedback dynamic (20%)


In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')
image_2048 = np.load('../Data/npy/Image_2048D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [4]:
print('user_following shape ',user_following.shape)
print('image_2048 shape ',image_2048.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (1489, 88)
image_2048 shape  (88, 2048)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)


In [5]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (1489, 17)


In [6]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
print(following_true)

[[13, 15, 16, 24, 29, 37, 44], [3, 25, 67, 71, 74, 75, 82], [8, 11, 24, 28, 41, 63, 67, 79, 86], [8, 24, 28, 29, 63, 70, 79], [8, 24, 37, 63, 67, 70, 79], [8, 17, 24, 25, 31, 43, 63, 67, 70, 73, 74, 79], [3, 10, 11, 22, 40, 44, 47, 74, 76, 82], [8, 17, 21, 27, 28, 32, 33, 45, 49, 67, 75, 82, 84], [3, 25, 63, 74, 79], [3, 10, 11, 14, 25, 40, 45, 54, 60, 65, 67, 74, 75, 79, 82], [3, 11, 22, 24, 40, 54, 57, 63, 67, 74, 76, 79], [7, 11, 14, 24, 25, 40, 43, 54, 63, 67, 75, 79], [7, 8, 24, 41, 63, 79], [0, 11, 12, 25, 26, 27, 32, 33, 42, 44, 45, 60, 67, 69, 71, 72, 74, 76, 79, 82, 84], [8, 24, 29, 43, 57, 63, 67, 76, 79], [3, 8, 25, 28, 40, 75, 79], [8, 54, 63, 67, 70, 79], [3, 9, 10, 25, 40, 44, 45, 54, 60, 67, 71, 74, 76, 82, 84], [8, 10, 28, 40, 43, 45, 54, 67, 76, 79], [25, 32, 33, 67, 71, 82, 83, 84], [8, 24, 43, 63, 67, 70, 79], [8, 24, 28, 29, 41, 44, 57, 79], [8, 14, 24, 29, 41, 43, 57, 63, 67, 78, 79], [0, 12, 26, 27, 32, 33, 53, 55, 72, 84], [12, 17, 18, 27, 32, 33, 43, 45, 56, 63,

In [7]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  34


In [8]:
over_10 = 0
over_8 = 0
over_12 = 0
for num in num_of_follower:
    if num >= 10:
        over_10 += 1
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)

over 8: 1073
over 10: 792
over 12: 499


# Training data and Testing data

In [9]:
test_amount = 150
yt_test_amount = 20

In [10]:
user_idx = [i for i in range(len(user_following))]
#test_idx is the number of user for testing
test_idx = random.sample(user_idx,test_amount)

In [11]:
# Training  and Testing
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        # random choose 2 true and 8 false for test 
        if len(temp_t)*0.2 < 1:
            t_for_test = random.sample(temp_t,1)
            f_for_test  = random.sample(temp_f,yt_test_amount-1)
        else:
            t_num = int(round(float(len(temp_t)*0.2)))
            t_for_test = random.sample(temp_t,t_num)
            f_for_test  = random.sample(temp_f,yt_test_amount-t_num)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [12]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/150
avg

2.0

In [13]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1489
The length of train_f: 1489
The length of test_t: 150
The length of test_f: 150


# Recommendation  Module

In [14]:
"""
n: the number of users
m: the number of YouTubers
k: latent dims
l: feature dims
"""
n = 1489
m = 88  
k = 128
l = 2048 

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None,l))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))


image_i = tf.placeholder(tf.float32, shape=(1,l))
image_j = tf.placeholder(tf.float32, shape=(1,l))

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [n, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    W1 = tf.get_variable("W1", [n, k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [k,k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [m,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [k,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
    
    

    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    Beta = tf.get_variable("beta", [n,l],
                             # initializer=tf.contrib.layers.xavier_initializer())
                                     initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i) 
vj = tf.nn.embedding_lookup(item_latent, j)

w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
wu = Wu
#wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
wa = Wa
#wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(k*k)
wv = Wv
#wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [15]:
a_list=tf.Variable([])
q = tf.constant(0)
def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])
def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    
    a_list = tf.concat([a_list,[(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
        tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q]]],0)
    q += 1
    return q,  a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

a_list_soft=tf.nn.softmax(a_list)


aux_np = tf.expand_dims(tf.zeros(128),0) #dimension (1,32)
q = tf.constant(0)
def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
    q += 1
    return q, aux_np

_,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

"""
for q in range(3): #取q個auxliary item
    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
"""


aux_np+=u #user_latent factor + sum (alpha*auxilary)
aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np

#矩陣中對應函數各自相乘
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,image_i, transpose_b=True)
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,image_j, transpose_b=True)

xuij = xui- xuj

l2_norm = tf.add_n([
            0.001 * tf.reduce_sum(tf.multiply(u, u)),
            0.001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.001 * tf.reduce_sum(tf.multiply(w1, w1)),
            0.001 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.001 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.001 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.001 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.1 * tf.reduce_sum(tf.multiply(beta,beta)),
            
          ])

loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

Instructions for updating:
Use `tf.cast` instead.


In [19]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0=time.time()

#use_true=init_list_of_objects(136)
#use_test=init_list_of_objects(136)

train_pair_t=[] #positive feedback
train_pair_f=[] #negative feedback
train_yes_id=[] 
for q in range(5):
    print('Iteraction:',q)
    train_auc=0
    total_loss=0
    xuij_auc=0
    length = 0
    for z in range(1489):
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes=[]
        yesr=[]
        
        
        sample=random.sample(train_t[z],len(train_t[z])) #隨機選3個sample true's YouTuber
        train_yes_id.append(sample) #sample全部丟進去
        
        #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
        #change
        r_3=np.zeros(len(sample)) 
        alpha_history = []
        a_list_history = []
        U_history = []
        Y_history = []
        
        #print(len(sample))
        #check if all YouTuber are in train_t or train_f
        #if len(train_t[z])+len(train_f[z]) != 88:
            #print(z,len(train_t[z])+len(train_f[z]))
         
        for k in range(len(sample)):
            yes.append(image_2048[sample[k]])
            yesr.append(YouTuber_category[sample[k]]*user_category_norm[z])
            #print('YouTuber_category ', YouTuber_category[sample[k]])
            #print('User_category ',user_category_norm[z])
        #print(len(yes))
        for k in range(len(sample)):
            r_3[k]=max(yesr[k])
        #print('r_3:',r_3)
        
        yes=np.array(yes)
        #print('user shape should be ',np.array([z]).shape)
        #print('xf shape should be ',yes.shape)
        #print('r shape should be ',np.array(r_3).shape)
        #print('l_id shape should be ',np.array(sample).shape)
        
        #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
        train_t_sample = random.sample(train_t[z],len(train_t[z]))
        #print('number of positive feedback', len(train_t[z]))
        for ta in train_t_sample:
            #ta=random.choice(train_t[z]) #ta is true positve photo
            train_pair_t.append(ta)
            image_1=np.expand_dims(image_2048[ta],0) #(1,2048)
            #print('Image_1 shape ',image_1.shape)
            train_f_sample = random.sample(train_f[z],20)
            for b in train_f_sample:
                #print('likes:',ta,';Not likes:',b)
                #b=random.choice(train_f[z])  #b is no feedback photo
                train_pair_f.append(b)
                image_2=np.expand_dims(image_2048[b],0) #(1,2048)
                #print('Image_2 shape',image_2.shape)
            
                #use_test[z].append(b)
                Uu,Yy,_a_list,r3,_auc, _loss,_=sess.run([user_latent,item_latent,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: yes , l_id:sample, l_id_len:[len(sample)],r:r_3,
                                        image_i:image_1,image_j:image_2})
                #print('User latent factor')
                #print(Uu.shape)
                #print(Uu)
                #print('Item latent factor')
                #print(Yy.shape)
                #print(Yy)
                U_history.append(Uu)
                Y_history.append(Yy)
                
                #print(XUIJ)
                #print('loss=',_loss)
                #print('auc=',_auc)
                #print(z,ta,b)
                #print('alpha list after softmax:',r3)
                #print('alpha list before softmax:',_a_list)
                a_list_history.append(_a_list)
                alpha_history.append(r3)
                train_auc+=_auc
                total_loss+=_loss
                length += 1
            #now1+=1
            
        with open('../Data/alpha_txt/'+str(q)+'_'+str(z)+'.txt', 'w') as f:
            for idd in range(len(alpha_history)):
                f.write('softmax alpha:')
                f.write(str(alpha_history[idd])+'\n')
                f.write('before softmax:')
                f.write(str(a_list_history[idd])+'\n')
        np.savez('../Data/latent_factor/'+str(q)+'_'+str(z)+'.npz', User=U_history, YouTuber=Y_history)
    
    #print('mine:',xuij_auc/136)    
    #print('a_list_soft:',r3)
    print("total_loss:-----------------", total_loss/length)
    print("train_auc:-------------------", train_auc/length)
    loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
    print('time:',time.time()-t0,' sec')
print('Total cost ',time.time()-t0,' sec')       

Iteraction: 0
total_loss:----------------- [[0.23338842]]
train_auc:------------------- 0.9376777408637874
time: 6585.584819555283  sec
Iteraction: 1
total_loss:----------------- [[0.15217773]]
train_auc:------------------- 0.9760863787375416
time: 15306.118017673492  sec
Iteraction: 2
total_loss:----------------- [[0.14251459]]
train_auc:------------------- 0.980704318936877
time: 24344.214306116104  sec
Iteraction: 3
total_loss:----------------- [[0.13951455]]
train_auc:------------------- 0.9820963455149502
time: 33117.7545003891  sec
Iteraction: 4
total_loss:----------------- [[0.13787766]]
train_auc:------------------- 0.9830431893687708
time: 42165.32394313812  sec
Total cost  42165.32394313812  sec


In [20]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('time=',loss_acc_list[i][2])

Iteration: 0
loss= [[0.23338842]]
acc= 0.9376777408637874
time= 6585.584819555283
Iteration: 1
loss= [[0.15217773]]
acc= 0.9760863787375416
time= 15306.118017673492
Iteration: 2
loss= [[0.14251459]]
acc= 0.980704318936877
time= 24344.214306116104
Iteration: 3
loss= [[0.13951455]]
acc= 0.9820963455149502
time= 33117.7545003891
Iteration: 4
loss= [[0.13787766]]
acc= 0.9830431893687708
time= 42165.32394313812


# Get latent factor and Each weight

In [21]:
U, Y, A, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, W1, Wu, Wy, Wa, Wv,Beta])

In [22]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)

User latent shape:  (1489, 128)
photo latent shape:  (88, 128)
Auxilary latent shape:  (88, 128)
W1 weight shape:  (1489, 128)
Wu weight shape: (128, 128)
Wy weight shape: (88, 128, 128)
Wa weight shape: (128, 128)
Wv weight shape: (128, 2048)
Beta shape: (1489, 2048)


# Testing Part

In [23]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au,np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa,
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av,np.expand_dims(image_2048[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    #print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], image_2048[k].T)
print(RS[s])

0 1484
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
1 1259
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
2 1191
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
3 1026
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
4 1465
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
5 1309
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
6 132
softmax alpha-------------- [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
7 1050
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
8 1116
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
9 1381
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
10 1192
softmax alpha-------------- [0.25 0.25 0.25 0.25]
11 1209
softmax alp

In [24]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [25]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

num of positive data in testing: 300.0
total testing data: 3000


In [26]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [27]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/total
print('avg_accuarcy for count_0:',avg_acc)

[19, 6, 0]
[0, 10]
[9, 0]
[5, 19, 16]
[1, 2, 10, 19]
[1, 16]
[1, 0]
[1, 0]
[9, 11]
[15, 0]
[0, 1]
[1, 0, 2]
[9]
[10]
[9]
[2, 6, 14]
[0]
[8, 11, 19]
[14, 8, 4, 9]
[3, 14]
[0, 2, 16]
[12]
[17, 6]
[10, 5]
[19, 11]
[1, 0]
[8, 2, 18]
[13, 8, 6, 9]
[2]
[2, 3]
[9, 18]
[19, 7, 10, 3]
[9, 17, 0]
[16]
[11, 17, 1]
[3, 17]
[0, 1]
[2, 12, 16, 3, 0]
[10, 0]
[5]
[5, 4, 16]
[9, 0]
[0]
[0, 5]
[15, 7]
[5]
[2, 0]
[1, 10]
[0]
[15, 19]
[3, 0]
[13, 9]
[3, 6]
[0]
[18, 5]
[1, 11, 0]
[1, 18]
[3, 6]
[2, 0]
[18, 0]
[10, 13]
[9, 1, 8]
[7, 1, 13]
[0]
[7, 4, 2, 3]
[19]
[9]
[1, 13]
[10]
[14, 19]
[3, 4, 13, 2]
[7, 0, 9]
[0, 16]
[6]
[0, 13]
[9]
[0, 19]
[12]
[17]
[0, 2]
[19]
[1, 0, 2]
[5, 10, 8]
[0, 1]
[5]
[0, 5, 2]
[4, 1]
[6]
[0, 1]
[18, 12]
[2, 1, 10]
[14]
[17, 15]
[15, 13]
[5]
[2, 19, 8]
[0, 17]
[5, 2, 0]
[3, 18]
[10, 11]
[12, 7]
[11]
[15]
[7]
[16]
[18, 7, 0]
[7]
[0, 15, 9]
[14]
[3]
[15, 1, 4]
[2, 0]
[3, 12, 2, 1]
[11, 12]
[7]
[12]
[18, 0, 1]
[0]
[1, 9]
[2, 1]
[4, 2]
[0, 2]
[7, 18]
[1, 4]
[2, 5, 16]
[14, 15]
[1, 8]


In [28]:
acc_0

101

In [30]:
total

714

# Top 1 

In [31]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [32]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

[19]
[0]
[9]
[5]
[1]
[1]
[1]
[1]
[9]
[15]
[0]
[1]
[9]
[10]
[9]
[2]
[0]
[8]
[14]
[3]
[0]
[12]
[17]
[10]
[19]
[1]
[8]
[13]
[2]
[2]
[9]
[19]
[9]
[16]
[11]
[3]
[0]
[2]
[10]
[5]
[5]
[9]
[0]
[0]
[15]
[5]
[2]
[1]
[0]
[15]
[3]
[13]
[3]
[0]
[18]
[1]
[1]
[3]
[2]
[18]
[10]
[9]
[7]
[0]
[7]
[19]
[9]
[1]
[10]
[14]
[3]
[7]
[0]
[6]
[0]
[9]
[0]
[12]
[17]
[0]
[19]
[1]
[5]
[0]
[5]
[0]
[4]
[6]
[0]
[18]
[2]
[14]
[17]
[15]
[5]
[2]
[0]
[5]
[3]
[10]
[12]
[11]
[15]
[7]
[16]
[18]
[7]
[0]
[14]
[3]
[15]
[2]
[3]
[11]
[7]
[12]
[18]
[0]
[1]
[2]
[4]
[0]
[7]
[1]
[2]
[14]
[1]
[0]
[19]
[0]
[8]
[0]
[3]
[3]
[1]
[14]
[4]
[7]
[18]
[7]
[16]
[4]
[16]
[18]
[4]
[0]
[13]
[19]
[5]
[7]


In [33]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.30666666666666664 recall  0.15333333333333332


In [36]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

F1_score: 0.20444444444444443


# Top 3

In [34]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

[19, 6, 0]
[0, 10, 1]
[9, 0, 1]
[5, 19, 16]
[1, 2, 10]
[1, 16, 0]
[1, 0, 8]
[1, 0, 12]
[9, 11, 6]
[15, 0, 10]
[0, 1, 18]
[1, 0, 2]
[9, 10, 6]
[10, 0, 16]
[9, 5, 12]
[2, 6, 14]
[0, 16, 2]
[8, 11, 19]
[14, 8, 4]
[3, 14, 8]
[0, 2, 16]
[12, 0, 15]
[17, 6, 0]
[10, 5, 12]
[19, 11, 16]
[1, 0, 2]
[8, 2, 18]
[13, 8, 6]
[2, 11, 0]
[2, 3, 6]
[9, 18, 6]
[19, 7, 10]
[9, 17, 0]
[16, 7, 19]
[11, 17, 1]
[3, 17, 15]
[0, 1, 16]
[2, 12, 16]
[10, 0, 14]
[5, 18, 8]
[5, 4, 16]
[9, 0, 16]
[0, 2, 5]
[0, 5, 14]
[15, 7, 12]
[5, 1, 10]
[2, 0, 16]
[1, 10, 7]
[0, 18, 9]
[15, 19, 14]
[3, 0, 17]
[13, 9, 17]
[3, 6, 18]
[0, 16, 7]
[18, 5, 3]
[1, 11, 0]
[1, 18, 17]
[3, 6, 2]
[2, 0, 1]
[18, 0, 15]
[10, 13, 15]
[9, 1, 8]
[7, 1, 13]
[0, 10, 11]
[7, 4, 2]
[19, 8, 3]
[9, 0, 6]
[1, 13, 14]
[10, 0, 6]
[14, 19, 11]
[3, 4, 13]
[7, 0, 9]
[0, 16, 7]
[6, 19, 2]
[0, 13, 1]
[9, 6, 3]
[0, 19, 15]
[12, 18, 0]
[17, 0, 10]
[0, 2, 1]
[19, 14, 5]
[1, 0, 2]
[5, 10, 8]
[0, 1, 15]
[5, 11, 10]
[0, 5, 2]
[4, 1, 13]
[6, 0, 17]
[0, 1, 11]
[18, 1

In [35]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.2644444444444444 recall  0.39666666666666667


In [37]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

F1_score: 0.31733333333333336


In [ ]:
F1_score(0.2222222222222222,0.0333333333333333)

# Top 5

In [38]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

[19, 6, 0, 14, 4]
[0, 10, 1, 16, 5]
[9, 0, 1, 17, 12]
[5, 19, 16, 12, 0]
[1, 2, 10, 19, 13]
[1, 16, 0, 6, 18]
[1, 0, 8, 19, 4]
[1, 0, 12, 2, 14]
[9, 11, 6, 14, 3]
[15, 0, 10, 11, 4]
[0, 1, 18, 7, 11]
[1, 0, 2, 3, 10]
[9, 10, 6, 3, 19]
[10, 0, 16, 18, 11]
[9, 5, 12, 17, 8]
[2, 6, 14, 1, 17]
[0, 16, 2, 6, 12]
[8, 11, 19, 5, 1]
[14, 8, 4, 9, 15]
[3, 14, 8, 6, 18]
[0, 2, 16, 1, 17]
[12, 0, 15, 17, 19]
[17, 6, 0, 1, 16]
[10, 5, 12, 7, 0]
[19, 11, 16, 14, 3]
[1, 0, 2, 5, 4]
[8, 2, 18, 5, 15]
[13, 8, 6, 9, 3]
[2, 11, 0, 10, 12]
[2, 3, 6, 13, 8]
[9, 18, 6, 7, 12]
[19, 7, 10, 3, 18]
[9, 17, 0, 19, 11]
[16, 7, 19, 3, 8]
[11, 17, 1, 0, 18]
[3, 17, 15, 13, 6]
[0, 1, 16, 18, 2]
[2, 12, 16, 3, 0]
[10, 0, 14, 1, 19]
[5, 18, 8, 17, 9]
[5, 4, 16, 10, 9]
[9, 0, 16, 19, 3]
[0, 2, 5, 8, 18]
[0, 5, 14, 2, 3]
[15, 7, 12, 0, 8]
[5, 1, 10, 4, 3]
[2, 0, 16, 14, 4]
[1, 10, 7, 12, 18]
[0, 18, 9, 10, 12]
[15, 19, 14, 0, 4]
[3, 0, 17, 6, 14]
[13, 9, 17, 4, 6]
[3, 6, 18, 19, 14]
[0, 16, 7, 10, 17]
[18, 5, 3, 4, 6]


In [39]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

prec  0.21466666666666667 recall  0.5366666666666666


In [40]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

F1_score: 0.3066666666666667


In [41]:
loaddata = np.load('../Data/latent_factor/0_0.npz')

In [42]:
User_latent_factor = loaddata['User']
YouTuber_latent_factor = loaddata['YouTuber']

In [45]:
print(User_latent_factor.shape)
print(YouTuber_latent_factor.shape)

(140, 1489, 128)
(140, 88, 128)


In [49]:
User_latent_factor[139]

array([[ 0.09262235, -0.07439136,  0.00155749, ..., -0.07487268,
        -0.02528663, -0.00655919],
       [-0.11952104, -0.00523345, -0.00117738, ...,  0.06385711,
         0.07218839, -0.13394912],
       [ 0.15034142,  0.03697095,  0.05438279, ..., -0.02960717,
         0.12263244,  0.0829374 ],
       ...,
       [-0.25210676,  0.11575705,  0.10584038, ..., -0.08173846,
        -0.03480363,  0.08626124],
       [-0.06635989,  0.19320491, -0.15399617, ...,  0.09256274,
        -0.0277265 , -0.04665166],
       [-0.04696163, -0.09584661,  0.03970387, ...,  0.00857876,
         0.13848732,  0.02843303]], dtype=float32)

In [50]:
YouTuber_latent_factor[139]

array([[ 1.05488598e-01, -8.71802345e-02,  1.18648242e-02, ...,
        -8.76639783e-02, -3.74541730e-02, -1.73222553e-02],
       [-1.20766789e-01, -4.02020710e-03,  1.30983084e-04, ...,
         6.51101023e-02,  7.34319538e-02, -1.32882565e-01],
       [ 1.49330139e-01,  3.79822180e-02,  5.53940684e-02, ...,
        -2.85958778e-02,  1.23643726e-01,  8.39484707e-02],
       ...,
       [-7.58756176e-02, -5.20650204e-03, -1.19225644e-01, ...,
        -3.81558798e-02,  7.09720924e-02,  3.08277160e-02],
       [ 2.59203613e-02, -2.47363211e-03,  7.64821097e-02, ...,
         1.28513249e-03, -9.45569798e-02, -1.27508372e-01],
       [-9.71664861e-02, -6.25763312e-02,  7.75660202e-02, ...,
         3.47987786e-02,  3.43698896e-02, -1.14809208e-01]], dtype=float32)